<a href="https://colab.research.google.com/github/Anasviel/Fake_news_detector/blob/main/main_fake_news_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git config --global user.name "Anasviel"
!git config --global user.email "kevinpages2002@gmail.com"
!git clone https://github.com/Anasviel/Fake_news_detector.git

Cloning into 'Fake_news_detector'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 5.16 KiB | 5.16 MiB/s, done.


In [2]:
%cd /content/Fake_news_detector/

/content/Fake_news_detector


In [3]:
!pwd
!ls

/content/Fake_news_detector
LICENSE  README.md


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:

df_welfake = pd.read_csv("WELFake_Dataset.csv")
df_custom = pd.read_csv("df_fake_true.csv", sep=";")
texts_to_predict = df_custom[df_custom["text"].notnull()]["text"].tolist()




In [ ]:

df_news = df_welfake
df_news.rename(columns={"label":"veracity"}, inplace=True)

random_state = 42
df_news = df_news.sample(frac=1).reset_index(drop=True)

In [ ]:
#Just to visualize the change and the blending of the two df
df_news.head(20)

,Unnamed: 0,title,text,veracity
0,50482,Re: What security threat? Huma Abedin doesn’t ...,"— Don't Panic (@dd42bb) October 30, 2016 @matt...",1
1,20019,Can the U.S. Really Get Astronauts to Mars by ...,In an essay published Tuesday on CNN’s website...,0
2,51967,Germany’s Extreme Right Challenges Guilt Over ...,"DRESDEN, Germany — At a beer hall on Tues...",0
3,52194,Risks ahead for U.S. tax push as disputes ling...,WASHINGTON (Reuters) - Congressional Republica...,0
4,23892,White House Floats 20 Percent Import Tax on Me...,President Donald Trump’s administration is flo...,0
5,55810,"‘Ghostbusters’ Is No. 2, Behind ‘Secret Life o...","LOS ANGELES — “We got one!” Those words, sh...",0
6,61698,Das Solowki-Archipel: Wo das Paradies auf die ...,26. Oktober 2016 Julia Shevelkina Russia Beyon...,1
7,62412,U.S. Says Russia Directed Hacks to Influence E...,WASHINGTON — The Obama administration on Fr...,0
8,27466,MINORITY TRUMP SUPPORTERS Thrown Out Of Maxine...,The video below begins with a Hispanic man tel...,1
9,35066,Trump Attacks The First Amendment Again With ...,Donald Trump has long had a contentious relati...,1


In [ ]:
stop_words = set(stopwords.words("english"))
stop_words.update(["reuters", "via", "said", "president donald", "follow", "twitter", "image",
    "november _num_", "october _num_", "com", "image via", "hillary",
    "october", "tuesday", "thursday", "thats", "dont", "washington", "friday",
    "wednesday", "monday", "obama", "_num__num__num_", "didnt", "there",
    "_num__num_", "he", "im", "obamas", "pic twitter", "sunday",
    "breitbart texas", "mr", "said statement", "pic", "saturday",
    "islamic state", "breitbart", "london", "womens", "follow twitter",
    "entire", "uk", "donate", "_num_the", "please", "president trump", "fbi",
    "share", "dc", "_num_yearold", "know", "wire", "however", "today",
    "century wire", "even", "rt", "post", "source"])
lemmatizer = WordNetLemmatizer()

In [ ]:
def clean_text_func(text):
  if not isinstance(text, str):
        return ""
  text = text.lower()
  text = re.sub(r'\d+', '_num_', text)
  text = re.sub(r'[^a-z0-9\s_]', '', text)
  words = nltk.word_tokenize(text)
  words = [lemmatizer.lemmatize(word, pos='n') for word in words if word not in stop_words]
  return ' '.join(words)

text_cleaning_transformer = FunctionTransformer(lambda texts: [clean_text_func(text) for text in texts], validate=False, accept_sparse=False)

def clean_text_no_numbers(text):
  if not isinstance(text, str):
        return ""
  text = text.lower()
  text = re.sub(r'\d+', '', text)
  text = re.sub(r'[^a-z0-9\s_]', '', text)
  words = nltk.word_tokenize(text)
  words = [lemmatizer.lemmatize(word, pos='n') for word in words if word not in stop_words]
  return ' '.join(words)

text_cleaning_transformer_no_numbers = FunctionTransformer(lambda texts: [clean_text_no_numbers(text) for text in texts], validate=False, accept_sparse=False)

In [ ]:
X = df_news["text"]
y = df_news["veracity"]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Original data shape : {X.shape}")
print(f"Training data shape : {X_train.shape}")
print(f"Testing data shape : {X_test.shape} \n")
print(f"Training labels distribution: \n {y_train.value_counts()}\n")
print(f"Testing labels distribution: \n {y_test.value_counts()}")

df_news["text"] = df_news["text"].fillna('')

Original data shape : (72134,)
Training data shape : (57707,)
Testing data shape : (14427,) 

Training labels distribution: 
 veracity
1    29685
0    28022
Name: count, dtype: int64

Testing labels distribution: 
 veracity
1    7421
0    7006
Name: count, dtype: int64


## Pipeline et randomgridsearch ayant servi à la selection du meilleur modèle et des hyperparamètres

---



In [ ]:
pipeline = Pipeline([
    ("cleaner", text_cleaning_transformer),
    ("tfidf", TfidfVectorizer()),
    ("classifier", MultinomialNB())
])


In [ ]:
"""param_grid = [
    {
        "cleaner":[text_cleaning_transformer, text_cleaning_transformer_no_numbers],
        "tfidf__max_features":[5000, 10000],
        "tfidf__ngram_range":[(1,1), (1,2)],
        "tfidf__smooth_idf":[True, False],
        "classifier":[MultinomialNB()],
        "classifier__alpha":[0.1, 0.5,1],
    },
    {
        "cleaner":[text_cleaning_transformer, text_cleaning_transformer_no_numbers],
        "tfidf__max_features":[5000, 10000],
        "tfidf__ngram_range":[(1,1), (1,2)],
        "tfidf__use_idf":[True, False],
        "classifier":[LogisticRegression(max_iter=1000, random_state=42)],
        "classifier__C":[0.1, 1, 2],
        "classifier__penalty":["l2"],
    },
    {
        "cleaner":[text_cleaning_transformer, text_cleaning_transformer_no_numbers],
        "tfidf__max_features":[5000, 10000],
        "tfidf__ngram_range":[(1,1), (1,2)],
        "tfidf__use_idf":[True, False],
        "classifier":[LinearSVC(max_iter=1000, random_state=42)],
        "classifier__C":[0.5, 1, 2],
    }]

SyntaxError: incomplete input (ipython-input-68-3482853318.py, line 1)

In [ ]:
"""scoring_metric = "f1_weighted"
random_search = RandomizedSearchCV(pipeline, param_grid, n_iter=10, cv=5, verbose=3, n_jobs=-1, scoring=scoring_metric, random_state=42)

print(f"Starting Random Search with {scoring_metric}")
sys.stdout.flush()
random_search.fit(X_train, y_train)
print("Random Search completed")
sys.stdout.flush()

print(f"\nBest Parameters found: {random_search.best_params_}")
print(f"\nBest Cross_Validation {scoring_metric} score: {random_search.best_score_:.2f}")

best_pipeline = random_search.best_estimator_
y_pred_best = best_pipeline.predict(X_test)

print(f"\nPerformance of the Best Model on the Test Set using {scoring_metric}:")
print(f"Accuracy:{accuracy_score(y_test, y_pred_best):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred_best, target_names=['True News', 'Fake News']))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_best))"""

##Ici on utilise un seul algorithme, linearSVC avec les paramètres les plus optimisés trouvés par randomgridsearch pour afficher les mots avec le plus de poids et donc résoudre le potentiel problème de data leakage.

In [ ]:
"""selected_cleaner = text_cleaning_transformer

# Définition du pipeline avec les meilleurs paramètres trouvés
model_for_leakage_test = Pipeline([
    ("cleaner", selected_cleaner),
    ("tfidf", TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
    ("classifier", LogisticRegression(C=1, max_iter=1000, random_state=random_state, solver="liblinear"))
])

print("\n--- Entraînement du modèle Logistc Regression avec les meilleurs paramètres trouvés ---")
model_for_leakage_test.fit(X_train, y_train)
print("Entraînement terminé.")

# --- Évaluation du modèle sur l'ensemble de test ---
y_pred = model_for_leakage_test.predict(X_test)

print(f"\nPerformance du modèle sur l'ensemble de Test :")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Fake News', 'True News']))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# --- ANALYSE DE LA FUITE DE DONNÉES / SURAPPRENTISSAGE avec LinearSVC ---
print("\n--- Analyse des coefficients du Logistic Regression pour détecter la fuite de données ---")

tfidf_vectorizer = model_for_leakage_test.named_steps['tfidf']
classifier = model_for_leakage_test.named_steps['classifier']

# Récupérer les noms des features (mots/n-grammes)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Récupérer les coefficients du classifieur
coefficients = classifier.coef_[0]

# Créer un DataFrame pour une meilleure visualisation
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients
})

# Ajouter une colonne pour la valeur absolue des coefficients (importance)
feature_importance['abs_coefficient'] = np.abs(feature_importance['coefficient'])

# Trier par importance absolue (les plus grands coefficients, qu'ils soient positifs ou négatifs)
feature_importance_sorted = feature_importance.sort_values(by='abs_coefficient', ascending=False)

print("\nTop 30 des mots/n-grammes les plus influents (toutes classes confondues):")
print(feature_importance_sorted.head(30))

# Afficher les mots les plus importants pour la classe "True News" (veracity=1)
# Les coefficients positifs sont pour la classe 1 (True News)
print("\nTop 30 des mots/n-grammes fortement associés aux Vraies Nouvelles (coef > 0):")
true_news_features = feature_importance.sort_values(by='coefficient', ascending=False)
print(true_news_features.head(30))

# Afficher les mots les plus importants pour la classe "Fake News" (veracity=0)
# Les coefficients négatifs sont pour la classe 0 (Fake News)
print("\nTop 30 des mots/n-grammes fortement associés aux Fausses Nouvelles (coef < 0):")
fake_news_features = feature_importance.sort_values(by='coefficient', ascending=True)
print(fake_news_features.head(30))"""

##Modèle Régression logistique


In [ ]:
logistic_regression_pipeline = Pipeline([
    ("cleaner", text_cleaning_transformer),
    ("tfidf", TfidfVectorizer(max_features=10000, ngram_range=(1,2))),
    ("classifier", LogisticRegression(C=0.1, max_iter=1000, random_state=random_state, solver="liblinear"))
])

print("\n---Entrainement du modèle Logistic Regression---")
logistic_regression_pipeline.fit(X_train, y_train)
print("Modèle Logistic Regression entraîné")

y_pred_test = logistic_regression_pipeline.predict(X_test)
y_proba_test = logistic_regression_pipeline.predict_proba(X_test)

print(f"\n---Performance du modèle sur le test set après correction des stop_words")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred_test, target_names=['Fake News', 'True News']))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))

new_texts = [
    "Scientists have found new but tentative evidence that a distant world orbiting another star could harbor life. A Cambridge team studying the atmosphere of a planet called K2-18b has detected signs of molecules that, on Earth, are only produced by simple organisms.This is the second, and most promising, time that chemicals associated with life have been detected in the planet's atmosphere by Nasa's James Webb Space Telescope (JWST). The team and independent astronomers stress, however, that further data is needed to confirm these results.",
    "A miracle that stunned the entire world: Three girls who went missing at a Texas summer camp were unexpectedly found alive conscious inside a hollow tree, less than a mile from the camp. After nearly ten days of facing hunger, cold, and confusion, the girls managed to survive thanks to two crucial survival tips. And as the story of these miracle survivors spread across social media, the community was moved to tears. No one expected that spokeswoman Karoline Leavitt would immediately take action",
    "Hackers have stolen personal information of a majority of insurance firm Allianz Life's 1.4 million customers in North America, its parent company said. On July 16, 2025, a malicious threat actor gained access to a third-party, cloud-based CRM system used by Allianz Life Insurance Company of North America (Allianz Life), Allianz said in a statement to the BBC.The German parent company added that the hackers were able to obtain personally identifiable data related to the majority of Allianz Life's customers, financial professionals, and select Allianz Life employees, using a social engineering technique. The data breach was only related to Allianz Life, according to the company.",
    "Israel has responded to sustained and growing international condemnation that it is responsible for starvation in Gaza by announcing a series of measures the Israel Defence Forces (IDF) said would 'improve the humanitarian response.' It is allowing airdrops of aid, carrying out the first one itself during the night and allowing the United Arab Emirates air force to follow with another later on Sunday. The IDF also announced that it would allow a 'tactical pause in military activity' in some areas and set up 'designated humanitarian corridors… to refute the false claim on international starvation.' Hamas has condemned the moves as a \"deception\". Israel, it said, was \"whitewashing its image before the world\"."
]

new_probabilities = logistic_regression_pipeline.predict_proba(texts_to_predict)
cleaned_new_texts = text_cleaning_transformer.transform(texts_to_predict)

for i, article in enumerate(texts_to_predict):
  proba_fake = new_probabilities[i][0] * 100
  proba_true = new_probabilities[i][1] * 100
  predicted_class = 0 if proba_fake > proba_true else 1
  predicted_veracity = "Faux article" if predicted_class == 0 else "Vrai article"

  print(f"\nArticle {i+1} :")
  print(f"Probabilité 'Faux article': {proba_fake:.2f}%")
  print(f"Probabilité 'Vrai article': {proba_true:.2f}%")
  print(f"Prédiction finale: {predicted_veracity}")
  print(f"Texte nettoyé: {cleaned_new_texts[i][:200]}...") # Utile pour debug


---Entrainement du modèle Logistic Regression---
Modèle Logistic Regression entraîné

---Performance du modèle sur le test set après correction des stop_words
Accuracy: 0.83
Classification Report:
              precision    recall  f1-score   support

   Fake News       0.86      0.77      0.82      7006
   True News       0.81      0.88      0.84      7421

    accuracy                           0.83     14427
   macro avg       0.83      0.83      0.83     14427
weighted avg       0.83      0.83      0.83     14427

Confusion Matrix:
[[5429 1577]
 [ 881 6540]]

Article 1 :
Probabilité 'Faux article': 54.88%
Probabilité 'Vrai article': 45.12%
Prédiction finale: Faux article
Texte nettoyé: french prime minister presented plan save _num_ billion euro among proposal patient longer bear brunt healthcare cost asked expire shortest possible time report must longer burden healthcare system re...

Article 2 :
Probabilité 'Faux article': 41.90%
Probabilité 'Vrai article': 58.10%
Prédiction fi

##Modèle MultinomialNB

In [ ]:
multinomialNB_pipeline = Pipeline([
    ("cleaner", text_cleaning_transformer),
    ("tfidf", TfidfVectorizer(max_features=10000, ngram_range=(1,2))),
    ("classifier", MultinomialNB(alpha=1, fit_prior=False, class_prior=[0.6, 0.4]))
])

print("\n---Entrainement du modèle Naive Bayes---")
multinomialNB_pipeline.fit(X_train, y_train)
print("Modèle Naive Bayes entraîné")
print(f"Ordre des classes : {multinomialNB_pipeline.named_steps['classifier'].classes_}")

y_pred_test = multinomialNB_pipeline.predict(X_test)
y_proba_test = multinomialNB_pipeline.predict_proba(X_test)

print(f"\n---Performance du modèle sur le test set après correction des stop_words")
print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred_test, target_names=['Fake News', 'True News']))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))

new_probabilities = multinomialNB_pipeline.predict_proba(texts_to_predict)
cleaned_new_texts = text_cleaning_transformer.transform(texts_to_predict)

for i, article in enumerate(texts_to_predict):
  proba_fake = new_probabilities[i][0] * 100
  proba_true = new_probabilities[i][1] * 100
  predicted_class = 0 if proba_fake > proba_true else 1
  predicted_veracity = "Faux article" if predicted_class == 0 else "Vrai article"

  print(f"\nArticle {i+1} :")
  print(f"Probabilité 'Faux article': {proba_fake:.2f}%")
  print(f"Probabilité 'Vrai article': {proba_true:.2f}%")
  print(f"Prédiction finale: {predicted_veracity}")
  print(f"Texte nettoyé: {cleaned_new_texts[i][:200]}...") # Utile pour debug


---Entrainement du modèle Naive Bayes---
Modèle Naive Bayes entraîné
Ordre des classes : [0 1]

---Performance du modèle sur le test set après correction des stop_words
Accuracy: 0.81
Classification Report:
              precision    recall  f1-score   support

   Fake News       0.79      0.84      0.81      7006
   True News       0.84      0.79      0.81      7421

    accuracy                           0.81     14427
   macro avg       0.81      0.81      0.81     14427
weighted avg       0.81      0.81      0.81     14427

Confusion Matrix:
[[5875 1131]
 [1565 5856]]

Article 1 :
Probabilité 'Faux article': 90.58%
Probabilité 'Vrai article': 9.42%
Prédiction finale: Faux article
Texte nettoyé: french prime minister presented plan save _num_ billion euro among proposal patient longer bear brunt healthcare cost asked expire shortest possible time report must longer burden healthcare system re...

Article 2 :
Probabilité 'Faux article': 20.14%
Probabilité 'Vrai article': 79.86%
Préd